In [ ]:
!pip install h2o

In [12]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

In [11]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,38 mins 14 secs
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_saeed_b4pt38
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.732 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [25]:
#churn_df = h2o.import_file('data/TelcoCustomerChurn.csv')
churn_df = h2o.import_file('../python/data/IMDB/ImdbTitleEpisode.csv')
#df = pd.read_csv('data/TelcoCustomerChurn.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [28]:
churn_df.types


{'tconst': 'string',
 'parentTconst': 'enum',
 'seasonNumber': 'int',
 'episodeNumber': 'int'}

In [29]:
churn_df.describe()

Rows:5652641
Cols:4

,tconst,parentTconst,seasonNumber,episodeNumber
type,string,enum,int,int
mins,NaN,,1.0,0.0
mean,NaN,,3.765361062357831,330.7354517805147
maxs,NaN,,2021.0,91334.0
sigma,NaN,,23.16946269373601,1214.7859766542576
zeros,0,,0,4331
missing,0,0,1156077,1156077
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,nan,nan


In [30]:
churn_train,churn_test,churn_valid = churn_df.split_frame(ratios=[.7, .15])

In [31]:
churn_train

tconst,parentTconst,seasonNumber,episodeNumber
tt0042816,tt0989125,1,17
tt0042889,tt0989125,nan,nan
tt0043710,tt0989125,3,3
tt0044093,tt0959862,1,6
tt0044901,tt0989125,3,46
tt0045519,tt0989125,4,11
tt0046135,tt0989125,4,5
tt0046855,tt0046643,1,4
tt0047810,tt0914702,3,36
tt0047852,tt0047745,1,15


In [32]:
y = "Churn"
x = churn_df.columns
x.remove(y)
x.remove("customerID")

ValueError: list.remove(x): x not in list

In [ ]:
aml = H2OAutoML(max_models = 10, seed = 10, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)


In [ ]:
!nvidia-smi

In [ ]:
aml.train(x = x, y = y, training_frame = churn_train, validation_frame=churn_valid)

In [ ]:
!nvidia-smi

In [ ]:
lb = aml.leaderboard

In [ ]:
lb.head()

In [ ]:
churn_pred=aml.leader.predict(churn_test)

In [ ]:
churn_pred.head()

In [ ]:
aml.leader.model_performance(churn_test)

In [ ]:
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
#se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
#metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
model_ids

In [ ]:
h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])

In [ ]:
out = h2o.get_model([mid for mid in model_ids if "XGBoost" in mid][0])

In [ ]:
out.params

In [ ]:
out.convert_H2OXGBoostParams_2_XGBoostParams()

In [ ]:
out

In [ ]:
out_gbm = h2o.get_model([mid for mid in model_ids if "GBM" in mid][0])

In [ ]:
out.confusion_matrix()

In [ ]:
out.varimp_plot()

In [ ]:
aml.leader.download_mojo(path = "./")